# Neural Networks in Tensorflow

In Tensorflow, a neural network is represented by a graph, which is sometimes called an Execution Graph. This is a graph in the traditional sense, that is, comprised of nodes and edges.

The nodes of the graph are instances of the `tensorflow.Variable` and `tensorflow.Operation` classes. The edges of the graph are represented by `tensorflow.Tensor` objects.

## Deferred Execution

Most computation takes place on the stack, as code is evaluated, and so is short-lived. Code is parsed and interpreted, execution occurs, results are stored, and the representation is disposed of.

Although this is generally an efficient way to specify computation, we do not always want to dispose of our computation as we go along. It is sometimes the case that we want to analyze our computation, or in other words, transform the computation itself, as data.

Whereas the first mode of evaluation (short lived, stack-based) is called **eager** evaluation, the mode of evaluation we desire is called **deferred**, or lazy evaluation.

But Python is an eager language. How then do we achieve deferred computation, when the Python interpreter is designed to eagerly evaluate all of our code? We do so by using **Objects**.

Objects are long-lived data structures. They do not reside in the stack, and so they do not disappear when computations are completed. They instead reside in the **heap**, and they effectively live forever (or until they are forgotten).

Thus, we represent deferred computations as instances of classes. Specifically, the topmost representation of a deferred computation in Tensorflow is a graph.

Graphs have many advantages.

1. A graph can be loaded, saved, and rebuilt at will.
2. Execution of a graph can be triggered at any time.
3. A graph can be submitted to different back-ends for processing (GPUs, TPUs, Mapreduce, etc).
4. Graphs can track the state of an iterative process (Memory reuse).

A graph is composed of nodes and edges. In tensorflow, the nodes are represented by instances of the `tensorflow.Variable` and `tensorflow.Operation` classes. The edges are represented by instances of the `tensorflow.Tensor` class.

In [1]:
import tensorflow

# Use revision 1 for now.
if int(tensorflow.__version__[0]) > 1:
    import tensorflow.compat.v1 as tensorflow
else:
    import tensorflow
    
tensorflow.disable_eager_execution()

Instantiate a new graph.

In [2]:
g = tensorflow.Graph()
g

## Variables and Operations as Nodes

The nodes of a Tensorflow graph are instances of `tensorflow.Variable` and `tensorflow.Operation`.

### Variables

The `tensorflow.Variable` class creates an object that tracks a single tensor.

A `tensorflow.Variable` object is created with a function that generates an *initial value*.

In [3]:
import random

v = tensorflow.Variable(random.random)
v

<tf.Variable 'Variable:0' shape=() dtype=float32>

Notice that the representation of `v` gives no indication of what its stored value is. This is because the initializer hasn't been run yet, and the variable is still *uninitialized*.

When we create `Variable` and `Operation` objects, they are always associated to a graph. If there is no graph explicitly declared, they are associated to a hidden, default graph.

In [4]:
v.graph

In [5]:
v.graph != g

True

If we want to associate a variable or object to a particular graph, we use the graph as a *context*.

In [6]:
# Declare v in the context of g.

with g.as_default():
    v = tensorflow.Variable(random.random)

v.graph == g

True

Aside: If your initializer is a constant function, you can simply pass the constant, and the backend will create an initializer function for it.

In [7]:
with g.as_default():
    u = tensorflow.Variable([1.0, 2.0, 3.0])
u

<tf.Variable 'Variable_1:0' shape=(3,) dtype=float32>

### Operations

An operation is a tensor mapping.

In [8]:
with g.as_default():
    scaled = tensorflow.math.scalar_mul(v, u)

scaled

<tf.Tensor 'Mul:0' shape=(3,) dtype=float32>

You may have expected `scaled` to be of type `tensorflow.Operation`. In fact, an operation is in 1-1 correspondence with its output tensor, so they are somewhat interchangeable. In the case of tensorflow, the tensor is returned, and the Operation is stored as an attribute of the tensor.

In [9]:
scaled.op

<tf.Operation 'Mul' type=Mul>

In [10]:
with g.as_default():
    normed = tensorflow.norm(scaled)

normed.op

<tf.Operation 'norm/Squeeze' type=Squeeze>

In Python, the arithmetic operators are overrideable. Because of this, tensorflow operators are built into the arithmetic ones, so you don't have to call, for instance, `tensorflow.math.scalar_mul`.

In [11]:
with g.as_default():
    scaled_again = v * u

scaled_again.op

<tf.Operation 'mul_1' type=Mul>

## Example: Changing from Polar to Cartesian Coordinates

The conversion from polar to cartesian coordinates is as follows:

$$
\begin{array}{rcl}
x &=& r \cdot \sin(\theta) \cdot \cos(\varphi)\\
y &=& r \cdot \sin(\theta) \cdot \sin(\varphi)\\
z &=& r \cdot \cos(\theta)
\end{array}
$$

In code:

In [12]:
sin = tensorflow.math.sin
cos = tensorflow.math.cos

def polar_to_cartesian(r, theta, phi):
    """Convert polar coordinates (r, theta, phi) to cartesian coordinates, (x, y, z)"""
    x = r * sin(theta) * cos(phi)
    y = r * sin(theta) * sin(phi)
    z = r * cos(theta)
    return (x, y, z)

## Visualization

We can use TensorBoard to visualize an execution graph.

Let's visualize the `polar_to_cartesian` execution graph. To do so, we'll create a new graph, and then write it using `tensorflow.summary.FileWriter`.

In [13]:
ptc_graph = tensorflow.Graph()

with ptc_graph.as_default():
    x = tensorflow.Variable(random.random)
    y = tensorflow.Variable(random.random)
    z = tensorflow.Variable(random.random)
    r, theta, phi = polar_to_cartesian(x,y,z)

In [14]:
tensorboard_dir = "tensorboard"

!mkdir -p {tensorboard_dir}
!rm {tensorboard_dir}/*

tensorflow.summary.FileWriter(tensorboard_dir, ptc_graph)

In [15]:
%load_ext tensorboard
%tensorboard --logdir {tensorboard_dir}

Reusing TensorBoard on port 6008 (pid 69023), started 10:10:30 ago. (Use '!kill 69023' to kill it.)

## Running a graph

We use a session to run a graph.

In [19]:
import math

with ptc_graph.as_default():
    
    # Give [r, theta, phi] some more predictable values
    r = tensorflow.Variable(1.0)
    theta = tensorflow.Variable(math.pi / 2.0)
    phi = tensorflow.Variable(math.pi)
    
    # Redefine [x,y,z] with the new variables
    x, y, z = polar_to_cartesian(r, theta, phi)
    
    with tensorflow.Session() as session:
        
        # Make sure all of the variables are initialized.
        session.run(tensorflow.global_variables_initializer())
        
        # You can pass a tensor of variables to session.run to evaluate them all at once.
        input_vector, output_vector = session.run(([x,y,z], [r,theta,phi]))
        
        print("Results:")
        print("  Input vector: [r, theta, phi] = %r" % output_vector)
        print("  Output vector: [x, y, z] = %r\n" % input_vector)  

Results:
  Input vector: [r, theta, phi] = [1.0, 1.5707964, 3.1415927]
  Output vector: [x, y, z] = [-1.0, -8.742278e-08, -4.371139e-08]

